In [2]:
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings

warnings.filterwarnings("ignore")
dataview_folder = 'JAQS_Data/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

Dataview loaded successfully.


In [3]:
def mask_index_member():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = ~(df_index_member ==0) #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
    # 跌停
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [4]:
mask = mask_index_member()
can_enter,can_exit = limit_up_down()
price = dv.get_ts('close_adj')
price_bench = dv.data_benchmark

Add formula failed: name [up_limit] exist. Try another name.
Add formula failed: name [down_limit] exist. Try another name.


In [9]:
# 优化因子
from jaqs.research import Optimizer
factor_opt = Optimizer(dataview=dv,
                      formula='- Correlation(vwap_adj, volume, LEN)',
                      params={"LEN":range(2,5,1)},
                      name='Cor_Vwap_Volume',
                      price=price,
                      benchmark_price=None,#=None求绝对收益 #=price_bench求相对收益
                      period=5,
                      n_quantiles=5,
                      mask=mask,
                      can_enter=can_enter,
                      can_exit=can_exit,
                      commission=0.0008,#手续费 默认0.0008
                      is_event=False,#是否是事件(0/1因子)
                      is_quarterly=False)#是否是季度因子 默认为False

In [10]:
factor_ic_opt = factor_opt.enumerate_optimizer(target_type="ic",
                                     target="IC Mean",
                                     in_sample_range=[20170101,20170701],
                                     ascending=False)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%


In [12]:
dict_opt = factor_opt.all_signals

In [22]:
perf_opt = factor_opt.cal_perf(dict_opt.get("Cor_Vwap_Volume{'LEN': 2}"))

In [23]:
perf_opt

{'ic':                        ic
 IC Mean      1.964567e-02
 IC Std.      8.345625e-02
 t-stat(IC)   8.954519e+00
 p-value(IC)  1.024009e-18
 IC Skew      1.070611e-01
 IC Kurtosis  2.671936e-01
 Ann. IR      2.354008e-01,
 'ret':            long_ret  short_ret  long_short_ret  top_quantile_ret  \
 t-stat          NaN        NaN       -1.687911         19.745207   
 p-value         NaN        NaN        0.091640          0.000000   
 skewness        NaN        NaN       -0.834367          0.168960   
 kurtosis        NaN        NaN       26.146432          5.825469   
 Ann. Ret        NaN        NaN       -0.044797          0.284545   
 Ann. Vol        NaN        NaN        0.145317          0.480125   
 Ann. IR         NaN        NaN       -0.308275          0.592647   
 occurance    1452.0     1452.0     1452.000000      53726.000000   
 
            bottom_quantile_ret      tmb_ret  
 t-stat               13.313658     5.681307  
 p-value               0.000000     0.000000  
 skewn

In [16]:
dict_opt.get("Cor_Vwap_Volume{'LEN': 2}")

signal    return  quantile
trade_date symbol                                     
20120105   000008.SZ  1.000000e+00  0.111011         5
           000031.SZ  1.000000e+00  0.073580         4
           000046.SZ  1.000000e+00  0.023770         5
           000400.SZ -1.000000e+00  0.027469         1
           000413.SZ -1.000000e+00  0.139239         2
           000415.SZ  1.000000e+00  0.143969         3
           000503.SZ -1.000000e+00  0.095019         1
           000539.SZ -1.000000e+00  0.023821         2
           000540.SZ -1.000000e+00  0.117621         1
           000555.SZ  1.000000e+00  0.080231         5
           000596.SZ -1.000000e+00  0.037004         2
           000598.SZ  1.000000e+00  0.114784         5
           000612.SZ -1.000000e+00  0.227828         3
           000627.SZ  1.000000e+00  0.056548         5
           000656.SZ -1.000000e+00  0.092833         2
           000671.SZ  1.000000e+00  0.060764         5
           000685.SZ -1.000000e+00  0.118055         1
           000690.SZ -1.000000e+00  0.101589         3
           000703.SZ -1.000000e+00  0.129689         2
           000712.SZ  1.000000e+00  0.054122         4
           000725.SZ  9.020467e-07  0.078468         3
           000738.SZ  1.000000e+00  0.085370         4
           000750.SZ -1.000000e+00  0.142769         1
           000793.SZ -1.000000e+00  0.036567         3
           000826.SZ  1.000000e+00  0.061978         5
           000883.SZ  1.000000e+00  0.146326         5
           000917.SZ  1.000000e+00  0.063149         4
           000927.SZ -1.000000e+00  0.066427         2
           000938.SZ -1.000000e+00  0.069342         3
           000951.SZ  1.000000e+00  0.082115         5
...                            ...       ...       ...
20171222   601016.SH  1.000000e+00  0.000000         5
           601098.SH  1.000000e+00  0.000000         5
           601101.SH -1.000000e+00  0.000000         2
           601106.SH -1.000000e+00  0.000000         3
           601107.SH  1.000000e+00  0.000000         5
           601127.SH -1.000000e+00  0.000000         2
           601139.SH -1.000000e+00  0.000000         1
           601158.SH -1.000000e+00  0.000000         1
           601168.SH  1.000000e+00  0.000000         5
           601179.SH  1.000000e+00  0.000000         5
           601231.SH -1.000000e+00  0.000000         2
           601233.SH  1.000000e+00  0.000000         4
           601238.SH -1.000000e+00  0.000000         1
           601258.SH  1.000000e+00  0.000000         5
           601369.SH -1.000000e+00  0.000000         3
           601519.SH -1.000000e+00  0.000000         1
           601558.SH -1.000000e+00  0.000000         1
           601566.SH -1.000000e+00  0.000000         3
           601666.SH -1.000000e+00  0.000000         2
           601699.SH -1.000000e+00  0.000000         3
           601717.SH  1.000000e+00  0.000000         4
           601808.SH -1.000000e+00  0.000000         1
           601918.SH  1.000000e+00  0.000000         4
           601928.SH  1.000000e+00  0.000000         4
           601929.SH -1.000000e+00  0.000000         3
           601969.SH -1.000000e+00  0.000000         1
           603000.SH  1.000000e+00  0.000000         4
           603288.SH  1.000000e+00  0.000000         5
           603699.SH -1.000000e+00  0.000000         3
           603885.SH -1.000000e+00  0.000000         1

[271544 rows x 3 columns]